# 📦 Automated Inventory Analysis and Reporting Platform

## Libraries Required

This project uses the following Python libraries:

### Core Data Processing & Visualization
- **streamlit** - Web application framework for creating interactive dashboards
- **pandas** - Data manipulation and analysis
- **numpy** - Numerical computing and array operations
- **matplotlib** - Data visualization and plotting

### Google API Integration
- **google-auth** - Google authentication library
- **google-api-python-client** - Google API client library

### Built-in Libraries (No Installation Required)
- **datetime** - Date and time handling
- **io** - Input/output operations
- **smtplib** - SMTP protocol client for sending emails
- **email** - Email message creation and handling
- **base64** - Base64 encoding/decoding

### Installation Command

Run the following command to install all required packages:

```bash
pip install streamlit pandas numpy matplotlib google-auth google-api-python-client openpyxl
```

**Note:** `openpyxl` is also required for reading/writing Excel files with pandas.

In [ ]:
import streamlit as st
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
import io
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.base import MIMEBase
from email import encoders
import base64
from google.oauth2.credentials import Credentials
from googleapiclient.discovery import build
from google.auth.transport.requests import Request

## 📋 Application Setup & Initialization

This section configures the Streamlit application and sets up the initial state:

### Page Configuration
```python
st.set_page_config(page_title="Warehouse Stock Analysis", layout="wide")
```
- Sets the browser tab title to "Warehouse Stock Analysis"
- Uses wide layout mode for better use of screen space

### Session State Management
```python
if 'df' not in st.session_state:
    st.session_state.df = None
if 'df2' not in st.session_state:
    st.session_state.df2 = None
if 'processed' not in st.session_state:
    st.session_state.processed = False
```
- Initializes session state variables to persist data across reruns
- `df` - Stores the Stock Source data
- `df2` - Stores the Fabric Stock data  
- `processed` - Tracks whether files have been uploaded and processed

### User Interface Elements
```python
st.title("Warehouse Stock Analysis Dashboard")

col1, col2 = st.columns(2)
with col1:
    stock_source_file = st.file_uploader("Upload Stock Source File", type=['xlsx'])
with col2:
    fabric_stock_file = st.file_uploader("Upload Fabric Stock File", type=['xlsx'])
```
- Displays the main dashboard title
- Creates two columns for file uploaders
- Accepts Excel files (.xlsx) only for both Stock Source and Fabric Stock data

In [ ]:
# Set page config
st.set_page_config(page_title="Warehouse Stock Analysis", layout="wide")

# Initialize session state
if 'df' not in st.session_state:
    st.session_state.df = None
if 'df2' not in st.session_state:
    st.session_state.df2 = None
if 'processed' not in st.session_state:
    st.session_state.processed = False

# Title
st.title("Warehouse Stock Analysis Dashboard")

# File upload section
col1, col2 = st.columns(2)
with col1:
    stock_source_file = st.file_uploader("Upload Stock Source File", type=['xlsx'])
with col2:
    fabric_stock_file = st.file_uploader("Upload Fabric Stock File", type=['xlsx'])


## 🔄 Data Processing Pipeline

This section processes the uploaded Excel files and performs comprehensive inventory analysis:

### 1️⃣ File Loading & Date Extraction
```python
df = pd.read_excel(stock_source_file, engine='openpyxl', sheet_name='Sheet')
df2 = pd.read_excel(fabric_stock_file, engine='openpyxl', sheet_name='Sheet')
```
- Loads both Stock Source and Fabric Stock files using openpyxl engine
- Extracts current date from fabric stock filename (e.g., "fabric stock 26-12-2024.xlsx")
- Falls back to current date if filename parsing fails

### 2️⃣ Warehouse Aggregation
```python
df_group1 = df.groupby('Warehouse')['Quantity'].sum().reset_index()
df2["one"] = 1
df_group2 = df2.groupby('Ware House')['one'].sum().reset_index()
```
- Aggregates stock quantities by warehouse location
- Adds a counter column to fabric stock for item counting
- Combines data from both sources including PF_Active warehouse

### 3️⃣ Warehouse Ordering
Defines a custom sort order for 18 warehouses based on production flow:
- **PF_Active (1)** - Fabric storage
- **WIP_Cut_1 (2)** - Cutting department
- **WIP_Pri_1 (3)** - Printing department  
- **WIP_Sew (5-8)** - Sewing departments
- **G_Active (11-12)** - Finished garments
- And more...

### 4️⃣ Time-Based Analysis
```python
df["number of days"] = (current_date - df['Last Movement Date']).dt.days
df["days cat"] = pd.cut(df["number of days"], bins=[-∞, 15, 30, 60, 90, 180, ∞])
```
Categorizes inventory by aging:
- **0-15 days** - Fresh stock
- **16-30 days** - Recent stock
- **31-60 days** - Aging stock
- **61-90 days** - Old stock
- **91-180 days** - Very old stock
- **180+ days** - Critical aging

### 5️⃣ Statistical Significance (Critical Items Detection)
Uses **Student's t-distribution** to identify statistically critical items:

```python
CI_pos = mean + t_value * (σ / √n)
df["Critical"] = df["number of days"] > CI_pos
```

**Method:**
- Calculates mean and standard deviation of days for each warehouse
- For n ≤ 30: Uses t-distribution (95% confidence interval)
- For n > 30: Uses normal distribution (z = 1.96)
- Items exceeding this threshold are marked as **Critical**

**Example:** If a warehouse averages 45 days with σ=10 and n=25, the critical threshold would be:
- CI = 45 + 2.060 × (10/√25) = 45 + 4.12 = **49.12 days**

### 6️⃣ Pivot Table Creation
```python
pivot_table = pd.pivot_table(df, values='Quantity', index='Warehouse', 
                             columns='days cat', aggfunc='sum')
```
- Creates a cross-tabulation of Warehouse × Time Categories
- Shows quantity distribution across aging periods for each location
- Combines stock source and fabric stock data

### 7️⃣ Summary Calculations
- **Time category totals** - Total quantities in each aging bucket
- **Critical totals** - Count of statistically critical items per warehouse
- **Ordered results** - All summaries sorted by warehouse production flow

### 8️⃣ Session State Storage
All processed data is saved to Streamlit session state:
- `df` & `df2` - Full processed datasets
- `df_grouped` - Warehouse quantity summaries
- `pivot_table` - Time distribution matrix
- `time_cat_totals` - Aging category totals
- `crucial_totals` - Critical items summary
- `current_date` - Analysis reference date
- `processed` - Processing completion flag

The app then reruns to display the dashboard with all analyzed data.

In [ ]:
# Process data when files are uploaded
if stock_source_file and fabric_stock_file and not st.session_state.processed:
    with st.spinner("Processing data..."):
        # Read files
        df = pd.read_excel(stock_source_file, engine='openpyxl', sheet_name='Sheet')
        df2 = pd.read_excel(fabric_stock_file, engine='openpyxl', sheet_name='Sheet')
        
        # Extract current date from fabric stock filename
        fabric_filename = fabric_stock_file.name
        try:
            date_part = fabric_filename.split('stock')[1].split('.')[0].strip()
            current_date = date_part.replace(' ', '-')
        except:
            current_date = datetime.now().strftime('%d-%m-%Y')
        
        # Aggregate by Warehouse
        df_group1 = df.groupby('Warehouse')['Quantity'].sum().reset_index()
        df2["one"] = 1
        df_group2 = df2.groupby('Ware House')['one'].sum().reset_index()
        
        # Add PF_Active row
        df_group1 = pd.concat([df_group1, df_group2[df_group2['Ware House'] == 'PF_Active'].rename(columns={'Ware House': 'Warehouse', 'one': 'Quantity'})], ignore_index=True)
        df_grouped = df_group1
        
        # Warehouse order
        warehouses = {'G_Active_1': 11, 'G_Active_2': 12, 'G_MD_1': 13, 'G_MD_2': 14, 
                      'HGBU_Extra': 18, 'Pre_Ship_1': 15, 'Pre_Ship_2': 16, 'WIPLines1': 9, 
                      'WIPLines2': 10, 'WIP_Cut_1': 2, 'WIP_Emb_1': 17, 'WIP_P1': 4, 
                      'WIP_Pri_1': 3, 'WIP_Sew_1': 5, 'WIP_Sew_2': 6, 'WIP_Sew_P1': 7, 
                      'WIP_Sew_P2': 8, 'PF_Active': 1}
        
        df_grouped['Order'] = df_grouped['Warehouse'].map(warehouses)
        df_grouped = df_grouped.sort_values(by='Order').reset_index(drop=True)
        
        # Calculate number of days
        df["number of days"] = (pd.to_datetime(current_date, format='%d-%m-%Y') - pd.to_datetime(df['Last Movement Date'], format='%d-%m-%Y')).dt.days
        df2["number of days"] = (pd.to_datetime(current_date, format='%d-%m-%Y') - pd.to_datetime(df2['last transaction date'], format='%d-%m-%Y')).dt.days
        
        # Days categories
        df["days cat"] = pd.cut(df["number of days"], bins=[-np.inf, 15, 30, 60, 90, 180, np.inf], 
                                labels=["0 - 15 days", "16 - 30 days", "31 - 60 days", "61 - 90 days", "91 - 180 days", "180+ days"])
        df2["days cat"] = pd.cut(df2["number of days"], bins=[-np.inf, 15, 30, 60, 90, 180, np.inf], 
                                 labels=["0 - 15 days", "16 - 30 days", "31 - 60 days", "61 - 90 days", "91 - 180 days", "180+ days"])
        
        # Statistical significance
        t_95_table = {2: 4.303, 3: 3.182, 4: 2.776, 5: 2.571, 6: 2.447, 7: 2.365, 8: 2.306, 
                      9: 2.262, 10: 2.228, 11: 2.201, 12: 2.179, 13: 2.160, 14: 2.145, 15: 2.131, 
                      16: 2.120, 17: 2.110, 18: 2.101, 19: 2.093, 20: 2.086, 21: 2.080, 22: 2.074, 
                      23: 2.069, 24: 2.064, 25: 2.060, 26: 2.056, 27: 2.052, 28: 2.048, 29: 2.045, 30: 2.042}
        
        statistical_sig = {}
        for i in df["Warehouse"].unique():
            temp_df = df[df["Warehouse"] == i]
            mean = temp_df["number of days"].mean()
            sigma = temp_df["number of days"].std()
            if len(temp_df) > 30:
                CI_pos = mean + 1.96 * (sigma / np.sqrt(len(temp_df)))
            else:
                CI_pos = mean + t_95_table[len(temp_df)] * (sigma / np.sqrt(len(temp_df)))
            statistical_sig[i] = CI_pos
        
        statistical_sig2 = {}
        for i in df2["Ware House"].unique():
            temp_df = df2[df2["Ware House"] == i]
            mean = temp_df["number of days"].mean()
            sigma = temp_df["number of days"].std()
            if len(temp_df) > 30:
                CI_pos = mean + 1.96 * (sigma / np.sqrt(len(temp_df)))
            else:
                CI_pos = mean + t_95_table[len(temp_df)] * (sigma / np.sqrt(len(temp_df)))
            statistical_sig2[i] = CI_pos
        
        df["Critical"] = df["number of days"] > df["Warehouse"].map(statistical_sig)
        df2["Critical"] = df2["number of days"] > df2["Ware House"].map(statistical_sig2)
        
        # Pivot tables
        pivot_table = pd.pivot_table(df, values='Quantity', index='Warehouse', columns='days cat', aggfunc='sum', fill_value=0)
        pivot_table2 = pd.pivot_table(df2, values='one', index='Ware House', columns='days cat', aggfunc='sum', fill_value=0)
        
        pivot_table = pd.concat([pivot_table, pivot_table2.loc[['PF_Active']].rename(index={'PF_Active': 'PF_Active'})], axis=0).fillna(0)
        pivot_table['Order'] = pivot_table.index.map(warehouses)
        pivot_table = pivot_table.sort_values(by='Order').drop(columns=['Order'])
        
        # Time category totals
        time_cat_totals = df.groupby('days cat')['Quantity'].sum().reset_index()
        
        # Critical totals
        crucial_totals = df[df['Critical']].groupby('Warehouse')['Quantity'].sum().reset_index()
        
        # Add PF_Active critical totals from df2
        if 'PF_Active' in df2['Ware House'].unique():
            pf_critical = df2[(df2['Ware House'] == 'PF_Active') & (df2['Critical'])].groupby('Ware House')['one'].sum().reset_index()
            if not pf_critical.empty:
                pf_critical.columns = ['Warehouse', 'Quantity']
                crucial_totals = pd.concat([crucial_totals, pf_critical], ignore_index=True)
        
        crucial_totals['Order'] = crucial_totals['Warehouse'].map(warehouses)
        crucial_totals = crucial_totals.sort_values(by='Order').drop(columns=['Order']).reset_index(drop=True)
        
        # Store in session state
        st.session_state.df = df
        st.session_state.df2 = df2
        st.session_state.df_grouped = df_grouped
        st.session_state.pivot_table = pivot_table
        st.session_state.time_cat_totals = time_cat_totals
        st.session_state.crucial_totals = crucial_totals
        st.session_state.current_date = current_date
        st.session_state.processed = True
        st.rerun()

## 📊 Main Dashboard Display

This section creates the interactive dashboard interface once data is processed:

### 🔍 Session State Retrieval
```python
df = st.session_state.df
df2 = st.session_state.df2
df_grouped = st.session_state.df_grouped
pivot_table = st.session_state.pivot_table
time_cat_totals = st.session_state.time_cat_totals
crucial_totals = st.session_state.crucial_totals
current_date = st.session_state.current_date
```
Loads all processed data from session state for display.

### 📅 Date Display
```python
st.info(f"Analysis Date: {current_date}")
```
Shows the analysis reference date extracted from the filename.

### 📦 Warehouse Summary Cards
```python
qty_sorted = df_grouped.sort_values(by='Quantity', ascending=False)
qty_sorted['Rank'] = qty_sorted.index + 1
```
**Features:**
- Creates a **6-column grid** of warehouse cards
- Ranks warehouses by total quantity (descending)
- **Top 3 warehouses** highlighted with red borders and background
- Other warehouses shown with standard blue styling
- Displays warehouse name and total quantity with comma formatting

**Visual Indicators:**
- 🔴 **Top 3**: Red border (#ff6b6b), pink background (#ffe0e0)
- 🔵 **Others**: Gray border, light blue background (#f0f8ff)

### 📊 Bar Chart Visualization
```python
fig, ax = plt.subplots(figsize=(12, 6), dpi=600)
ax.bar(df_grouped['Warehouse'], df_grouped['Quantity'], color='skyblue')
```
**Specifications:**
- High-resolution chart (600 DPI) for crisp display
- 12×6 inch figure size
- Sky blue bars for visual clarity
- 45° rotated warehouse labels for readability
- Automatic tight layout to prevent label clipping

### 📋 Pivot Table Display
```python
st.dataframe(pivot_table, use_container_width=True)
```
Interactive table showing **Warehouse × Time Category** matrix:
- Rows: All warehouses (ordered by production flow)
- Columns: 6 aging categories (0-15 days through 180+ days)
- Values: Total quantities in each category
- Full-width display for better readability

### 🎛️ Sidebar Filters
Creates two interactive filter controls:

**1. Filter Type** (Radio buttons)
- **Days** - Filter by time-based aging categories
- **Statistical** - Filter by statistically critical items

**2. Warehouse Selector** (Dropdown)
- **All** - View all warehouses combined
- Individual warehouse options from the processed data

These filters dynamically update the detailed items view and control what data is shown in the main content area.

In [ ]:
# Main dashboard
if st.session_state.processed:
    df = st.session_state.df
    df2 = st.session_state.df2
    df_grouped = st.session_state.df_grouped
    pivot_table = st.session_state.pivot_table
    time_cat_totals = st.session_state.time_cat_totals
    crucial_totals = st.session_state.crucial_totals
    current_date = st.session_state.current_date
    
    # Display current date
    st.info(f"Analysis Date: {current_date}")
    
    # Cards for total quantities by warehouse
    st.subheader("Total Quantity by Warehouse")
    qty_sorted = df_grouped.sort_values(by='Quantity', ascending=False).reset_index(drop=True)
    qty_sorted['Rank'] = qty_sorted.index + 1
    
    # Create cards using Streamlit columns
    num_cols = 6
    cols = st.columns(num_cols)
    for index, row in df_grouped.iterrows():
        col_idx = index % num_cols
        rank = qty_sorted.loc[qty_sorted['Warehouse'] == row['Warehouse'], 'Rank'].values[0]
        is_top3 = rank <= 3
        
        with cols[col_idx]:
            if is_top3:
                st.markdown(f"""
                <div style="border: 2px solid #ff6b6b; border-radius: 8px; padding: 12px; text-align: center; background-color: #ffe0e0;">
                    <p style="margin: 0; font-weight: bold; font-size: 14px;">{row['Warehouse']}</p>
                    <p style="margin: 5px 0 0 0; font-size: 20px; font-weight: bold; color: #d63031;">{int(row['Quantity']):,}</p>
                </div>
                """, unsafe_allow_html=True)
            else:
                st.markdown(f"""
                <div style="border: 1px solid #ccc; border-radius: 8px; padding: 12px; text-align: center; background-color: #f0f8ff;">
                    <p style="margin: 0; font-weight: bold; font-size: 14px;">{row['Warehouse']}</p>
                    <p style="margin: 5px 0 0 0; font-size: 20px; font-weight: bold; color: #2c3e50;">{int(row['Quantity']):,}</p>
                </div>
                """, unsafe_allow_html=True)
    
    # Bar chart
    st.subheader("Total Quantity Distribution")
    fig, ax = plt.subplots(figsize=(12, 6), dpi=600)
    ax.bar(df_grouped['Warehouse'], df_grouped['Quantity'], color='skyblue')
    ax.set_xlabel('Warehouse')
    ax.set_ylabel('Total Quantity')
    ax.set_title('Total Quantity by Warehouse')
    plt.xticks(rotation=45)
    plt.tight_layout()
    st.pyplot(fig, dpi=600)
    
    # Pivot table
    st.subheader("Quantity Distribution by Time Category")
    st.dataframe(pivot_table, use_container_width=True)
    
    # Sidebar filters
    st.sidebar.header("Filters")
    filter_type = st.sidebar.radio("Filter Type", ["Days", "Statistical"])
    
    warehouse_list = ['All'] + list(df_grouped['Warehouse'].unique())
    selected_warehouse = st.sidebar.selectbox("Select Warehouse", warehouse_list)


## 🔍 Detailed Items View & Filtering

This section provides a two-column layout for detailed inventory analysis with filtering options:

### 📐 Layout Structure
```python
col_left, col_right = st.columns([2, 1])
```
- **Left column (66%)** - Detailed items table with download functionality
- **Right column (33%)** - Summary cards and filter controls

---

## 📊 Right Column - Filter-Based Summaries

### 🗓️ Days Filter Mode
When "Days" filter is selected:

**Time Category Summary Cards**
```python
filtered_time_cats = df[df['Warehouse'] == selected_warehouse].groupby('days cat')['Quantity'].sum()
```
- Shows 6 cards for aging categories (0-15 days through 180+ days)
- **Styling**: Gray border, lime green background (#f0f4c3)
- Displays total quantities per category
- Adapts to selected warehouse (All or specific)

**Multi-Select Filter**
```python
selected_days = st.multiselect("Select Days Categories", days_categories, default=all)
```
- Allows selection of multiple time categories
- Default: All categories selected
- Updates the detailed items table dynamically

### 📈 Statistical Filter Mode
When "Statistical" filter is selected:

**Critical Items Summary Cards**
```python
display_crucial = crucial_totals[crucial_totals['Warehouse'] == selected_warehouse]
```
- Shows cards for warehouses with critical items
- **Styling**: Orange border (#ff9800), orange background (#ffe0b2)
- Displays count of statistically critical items per warehouse
- Filtered by selected warehouse

---

## 📋 Left Column - Detailed Items Table

### 🗓️ Days Filter - Detailed View

**Scenario 1: All Warehouses**
```python
filtered_df = df[df['days cat'].isin(selected_days)]
display_df = filtered_df[['Project', 'Color', 'Size', 'Quantity', 'Customer', 'Warehouse']]
```
- Shows items from **all warehouses** matching selected day categories
- **Columns**: Project, Color, Size, Quantity, Customer, Warehouse
- 400px height scrollable table
- **Download**: CSV format with filename `All_Warehouses_{date}.csv`

**Scenario 2: PF_Active Warehouse**
```python
filtered_df2 = df2[(df2['Ware House'] == 'PF_Active') & (df2['days cat'].isin(selected_days))]
display_df2 = filtered_df2[['Project', 'Lot No', 'Style-color', 'Gramaj']]
```
- Shows fabric stock items only
- **Columns**: Project, Lot No, Style-color, Gramaj (different schema for fabric)
- **Download**: CSV format with filename `PF_Active_{date}.csv`

**Scenario 3: Specific Warehouse**
```python
filtered_df = df[(df['Warehouse'] == selected_warehouse) & (df['days cat'].isin(selected_days))]
display_df = filtered_df[['Project', 'Color', 'Size', 'Quantity', 'Customer']]
```
- Shows items from selected warehouse only
- **Columns**: Project, Color, Size, Quantity, Customer (no Warehouse column)
- **Download**: CSV format with filename `{warehouse}_{date}.csv`

### 📈 Statistical Filter - Critical Items View

**Scenario 1: All Warehouses - Critical**
```python
filtered_df = df[df['Critical']]
display_df = filtered_df[['Project', 'Color', 'Size', 'Quantity', 'Customer', 'Warehouse']]
```
- Shows **all critical items** across all warehouses
- Items exceeding statistical threshold (95% CI)
- **Download**: CSV format with filename `Critical_All_Warehouses_{date}.csv`

**Scenario 2: PF_Active - Critical**
```python
filtered_df2 = df2[(df2['Ware House'] == 'PF_Active') & (df2['Critical'])]
```
- Shows critical fabric stock items
- Same column structure as Days filter for PF_Active
- **Download**: CSV format with filename `Critical_PF_Active_{date}.csv`

**Scenario 3: Specific Warehouse - Critical**
```python
filtered_df = df[(df['Warehouse'] == selected_warehouse) & (df['Critical'])]
```
- Shows critical items from selected warehouse
- Same column structure as Days filter for specific warehouse
- **Download**: CSV format with filename `Critical_{warehouse}_{date}.csv`

---

## 📥 Download Functionality

All scenarios include a download button:
```python
csv = display_df.to_csv(index=False).encode('utf-8')
st.download_button(label="📥 Download Current View", data=csv, ...)
```

**Features:**
- Exports currently displayed data as CSV
- Filename includes warehouse name, date, and filter type
- No index column in output
- UTF-8 encoding for international characters

---

## ℹ️ Empty State Handling

When no items match the criteria:
- **Days Filter**: "No items found for the selected day categories."
- **Statistical Filter**: "No critical items found."

Displays an info message instead of empty table.

In [ ]:
if st.session_state.processed:    
    # Right side content based on filter
    col_left, col_right = st.columns([2, 1])
    
    with col_right:
        if filter_type == "Days":
            st.subheader("Time Category Summary")
            
            # Calculate time category totals based on selected warehouse
            if selected_warehouse == 'All':
                filtered_time_cats = time_cat_totals
            else:
                if selected_warehouse == 'PF_Active':
                    filtered_time_cats = df2[df2['Ware House'] == selected_warehouse].groupby('days cat')['one'].sum().reset_index()
                    filtered_time_cats.columns = ['days cat', 'Quantity']
                else:
                    filtered_time_cats = df[df['Warehouse'] == selected_warehouse].groupby('days cat')['Quantity'].sum().reset_index()
            
            # Cards for time categories using columns
            for index, row in filtered_time_cats.iterrows():
                st.markdown(f"""
                <div style="border: 1px solid #9e9e9e; border-radius: 8px; padding: 12px; margin: 8px 0; text-align: center; background-color: #f0f4c3;">
                    <p style="margin: 0; font-weight: bold; font-size: 14px;">{row['days cat']}</p>
                    <p style="margin: 5px 0 0 0; font-size: 20px; font-weight: bold; color: #33691e;">{int(row['Quantity']):,}</p>
                </div>
                """, unsafe_allow_html=True)
            
            # Days category filter
            days_categories = ["0 - 15 days", "16 - 30 days", "31 - 60 days", "61 - 90 days", "91 - 180 days", "180+ days"]
            selected_days = st.multiselect("Select Days Categories", days_categories, default=days_categories)
            
        else:  # Statistical
            st.subheader("Critical Items Summary")
            
            # Cards for critical quantities
            display_crucial = crucial_totals if selected_warehouse == 'All' else crucial_totals[crucial_totals['Warehouse'] == selected_warehouse]
            for index, row in display_crucial.iterrows():
                st.markdown(f"""
                <div style="border: 1px solid #ff9800; border-radius: 8px; padding: 12px; margin: 8px 0; text-align: center; background-color: #ffe0b2;">
                    <p style="margin: 0; font-weight: bold; font-size: 14px;">{row['Warehouse']}</p>
                    <p style="margin: 5px 0 0 0; font-size: 20px; font-weight: bold; color: #e65100;">{int(row['Quantity']):,}</p>
                </div>
                """, unsafe_allow_html=True)
    
    with col_left:
        st.subheader("Detailed Items")
        
        if filter_type == "Days":
            # Filter by warehouse and days
            if selected_warehouse == 'All':
                filtered_df = df[df['days cat'].isin(selected_days)]
                if not filtered_df.empty:
                    st.write("**Stock Source (All Warehouses)**")
                    display_df = filtered_df[['Project', 'Color', 'Size', 'Quantity', 'Customer', 'Warehouse']].copy()
                    st.dataframe(display_df, use_container_width=True, height=400)
                    
                    # Download button
                    csv = display_df.to_csv(index=False).encode('utf-8')
                    st.download_button(
                        label="📥 Download Current View",
                        data=csv,
                        file_name=f"All_Warehouses_{current_date}.csv",
                        mime="text/csv"
                    )
                else:
                    st.info("No items found for the selected day categories.")
            else:
                if selected_warehouse == 'PF_Active':
                    filtered_df2 = df2[(df2['Ware House'] == selected_warehouse) & (df2['days cat'].isin(selected_days))]
                    if not filtered_df2.empty:
                        st.write("**Fabric Stock (PF_Active)**")
                        display_df2 = filtered_df2[['Project', 'Lot No', 'Style-color', 'Gramaj']].copy()
                        st.dataframe(display_df2, use_container_width=True, height=400)
                        
                        # Download button
                        csv = display_df2.to_csv(index=False).encode('utf-8')
                        st.download_button(
                            label="📥 Download Current View",
                            data=csv,
                            file_name=f"{selected_warehouse}_{current_date}.csv",
                            mime="text/csv"
                        )
                    else:
                        st.info("No items found for the selected day categories.")
                else:
                    filtered_df = df[(df['Warehouse'] == selected_warehouse) & (df['days cat'].isin(selected_days))]
                    if not filtered_df.empty:
                        st.write(f"**Stock Source ({selected_warehouse})**")
                        display_df = filtered_df[['Project', 'Color', 'Size', 'Quantity', 'Customer']].copy()
                        st.dataframe(display_df, use_container_width=True, height=400)
                        
                        # Download button
                        csv = display_df.to_csv(index=False).encode('utf-8')
                        st.download_button(
                            label="📥 Download Current View",
                            data=csv,
                            file_name=f"{selected_warehouse}_{current_date}.csv",
                            mime="text/csv"
                        )
                    else:
                        st.info("No items found for the selected day categories.")
        
        else:  # Statistical
            # Filter by critical items
            if selected_warehouse == 'All':
                filtered_df = df[df['Critical']]
                if not filtered_df.empty:
                    st.write("**Critical Items (All Warehouses)**")
                    display_df = filtered_df[['Project', 'Color', 'Size', 'Quantity', 'Customer', 'Warehouse']].copy()
                    st.dataframe(display_df, use_container_width=True, height=400)
                    
                    # Download button
                    csv = display_df.to_csv(index=False).encode('utf-8')
                    st.download_button(
                        label="📥 Download Critical Items",
                        data=csv,
                        file_name=f"Critical_All_Warehouses_{current_date}.csv",
                        mime="text/csv"
                    )
                else:
                    st.info("No critical items found.")
            else:
                if selected_warehouse == 'PF_Active':
                    filtered_df2 = df2[(df2['Ware House'] == selected_warehouse) & (df2['Critical'])]
                    if not filtered_df2.empty:
                        st.write("**Critical Fabric Stock (PF_Active)**")
                        display_df2 = filtered_df2[['Project', 'Lot No', 'Style-color', 'Gramaj']].copy()
                        st.dataframe(display_df2, use_container_width=True, height=400)
                        
                        # Download button
                        csv = display_df2.to_csv(index=False).encode('utf-8')
                        st.download_button(
                            label="📥 Download Critical Items",
                            data=csv,
                            file_name=f"Critical_{selected_warehouse}_{current_date}.csv",
                            mime="text/csv"
                        )
                    else:
                        st.info("No critical items found.")
                else:
                    filtered_df = df[(df['Warehouse'] == selected_warehouse) & (df['Critical'])]
                    if not filtered_df.empty:
                        st.write(f"**Critical Items ({selected_warehouse})**")
                        display_df = filtered_df[['Project', 'Color', 'Size', 'Quantity', 'Customer']].copy()
                        st.dataframe(display_df, use_container_width=True, height=400)
                        
                        # Download button
                        csv = display_df.to_csv(index=False).encode('utf-8')
                        st.download_button(
                            label="📥 Download Critical Items",
                            data=csv,
                            file_name=f"Critical_{selected_warehouse}_{current_date}.csv",
                            mime="text/csv"
                        )
                    else:
                        st.info("No critical items found.")

## 📦 Bulk Download & Email Reporting

This section provides advanced export and email notification features:

---

## 🗜️ Download All Warehouses (ZIP Archive)

### Sidebar Bulk Export Button
```python
if st.sidebar.button("📦 Download All Warehouses (ZIP)"):
    import zipfile
    from io import BytesIO
```

**Functionality:**
Creates a ZIP archive containing CSV files for **all warehouses** with current filters applied.

### ZIP File Generation Process

**Days Filter Mode:**
```python
if filter_type == "Days":
    for warehouse in df['Warehouse'].unique():
        warehouse_df = df[(df['Warehouse'] == warehouse) & (df['days cat'].isin(selected_days))]
```
- Exports each warehouse separately
- Applies selected day categories filter
- **Stock Source Columns**: Project, Color, Size, Quantity, Customer, Last Movement Date, number of days
- **Fabric Stock Columns**: Project, Lot No, Style-color, Gramaj, last transaction date, number of days
- **Filename Format**: `{warehouse}_{date}_DaysFilter.csv`

**Statistical Filter Mode:**
```python
else:  # Statistical filter
    warehouse_df = df[(df['Warehouse'] == warehouse) & (df['Critical'])]
```
- Exports only critical items per warehouse
- Same column structure as Days filter
- **Filename Format**: `{warehouse}_{date}_Critical.csv`

### Download Mechanism
```python
zip_buffer.seek(0)
st.sidebar.download_button(
    label="💾 Download ZIP File",
    data=zip_buffer,
    file_name=f"All_Warehouses_{current_date}_{filter_suffix}.zip"
)
```
- Uses in-memory buffer (no temporary files)
- ZIP compression (ZIP_DEFLATED algorithm)
- Dynamic filename based on date and filter type
- Empty warehouses excluded automatically

---

## 🔄 Reset Functionality

### Session State Clear & Reload
```python
if st.sidebar.button("🔄 Reset and Upload New Files"):
    for key in st.session_state.keys():
        del st.session_state[key]
    st.rerun()
```

**Purpose:**
- Clears all processed data from session state
- Resets application to initial state
- Allows uploading new files without page refresh
- Useful for analyzing different datasets

---

## 📧 Email Reporting System

### Department-to-Warehouse Mapping

Defines organizational structure for targeted email reports:

```python
department_warehouse_mapping = {
    "Garment Active (G_Active)": {
        "email": "garment.active@company.com",
        "warehouses": ['G_Active_1', 'G_Active_2']
    },
    ...
}
```

**Department Breakdown:**

| Department | Warehouses | Purpose |
|-----------|------------|---------|
| **Garment Active** | G_Active_1, G_Active_2 | Finished garment storage |
| **Garment MD** | G_MD_1, G_MD_2 | Merchandise distribution |
| **Pre-Shipment** | Pre_Ship_1, Pre_Ship_2 | Ready for shipping |
| **WIP Lines** | WIPLines1, WIPLines2 | Work-in-progress lines |
| **WIP Sewing** | WIP_Sew_1/2, WIP_Sew_P1/P2 | Sewing departments |
| **WIP Cutting & Print** | WIP_Cut_1, WIP_Pri_1, WIP_P1 | Cutting & printing |
| **WIP Embroidery** | WIP_Emb_1 | Embroidery department |
| **Fabric Department** | PF_Active | Fabric storage |
| **HGBU Extra** | HGBU_Extra | Extra storage |
| **All Warehouses** | All 18 warehouses | Management overview |

**Dynamic All Warehouses List:**
```python
"warehouses": list(df['Warehouse'].unique()) + 
              (['PF_Active'] if 'PF_Active' in df2['Ware House'].unique() else [])
```
- Automatically includes all warehouses from current data
- Adds PF_Active if fabric stock data exists

### Email Interface Layout
```python
col_email1, col_email2 = st.columns([1, 1])
```
- **Left column (50%)** - Email configuration and settings
- **Right column (50%)** - Files to be sent preview and statistics

This creates a balanced layout for email composition and attachment review.

In [ ]:
if st.session_state.processed:    
    # Download all warehouses button
    st.sidebar.markdown("---")
    st.sidebar.subheader("Download All Warehouses")
    if st.sidebar.button("📦 Download All Warehouses (ZIP)"):
        import zipfile
        from io import BytesIO
        
        zip_buffer = BytesIO()
        with zipfile.ZipFile(zip_buffer, 'w', zipfile.ZIP_DEFLATED) as zip_file:
            # Apply filters based on filter type
            if filter_type == "Days":
                # Export each warehouse from df with day filter applied
                for warehouse in df['Warehouse'].unique():
                    warehouse_df = df[(df['Warehouse'] == warehouse) & (df['days cat'].isin(selected_days))][['Project', 'Color', 'Size', 'Quantity', 'Customer', 'Last Movement Date', 'number of days']].copy()
                    if not warehouse_df.empty:
                        csv_data = warehouse_df.to_csv(index=False)
                        zip_file.writestr(f"{warehouse}_{current_date}_DaysFilter.csv", csv_data)
                
                # Export PF_Active if exists with day filter
                if 'PF_Active' in df2['Ware House'].unique():
                    pf_active_df = df2[(df2['Ware House'] == 'PF_Active') & (df2['days cat'].isin(selected_days))][['Project', 'Lot No', 'Style-color', 'Gramaj', 'last transaction date', 'number of days']].copy()
                    if not pf_active_df.empty:
                        csv_data = pf_active_df.to_csv(index=False)
                        zip_file.writestr(f"PF_Active_{current_date}_DaysFilter.csv", csv_data)
            
            else:  # Statistical filter
                # Export each warehouse from df with critical filter applied
                for warehouse in df['Warehouse'].unique():
                    warehouse_df = df[(df['Warehouse'] == warehouse) & (df['Critical'])][['Project', 'Color', 'Size', 'Quantity', 'Customer', 'Last Movement Date', 'number of days']].copy()
                    if not warehouse_df.empty:
                        csv_data = warehouse_df.to_csv(index=False)
                        zip_file.writestr(f"{warehouse}_{current_date}_Critical.csv", csv_data)
                
                # Export PF_Active if exists with critical filter
                if 'PF_Active' in df2['Ware House'].unique():
                    pf_active_df = df2[(df2['Ware House'] == 'PF_Active') & (df2['Critical'])][['Project', 'Lot No', 'Style-color', 'Gramaj', 'last transaction date', 'number of days']].copy()
                    if not pf_active_df.empty:
                        csv_data = pf_active_df.to_csv(index=False)
                        zip_file.writestr(f"PF_Active_{current_date}_Critical.csv", csv_data)
        
        zip_buffer.seek(0)
        filter_suffix = "DaysFilter" if filter_type == "Days" else "Critical"
        st.sidebar.download_button(
            label="💾 Download ZIP File",
            data=zip_buffer,
            file_name=f"All_Warehouses_{current_date}_{filter_suffix}.zip",
            mime="application/zip"
        )
    
    # Reset button
    st.sidebar.markdown("---")
    if st.sidebar.button("🔄 Reset and Upload New Files"):
        for key in st.session_state.keys():
            del st.session_state[key]
        st.rerun()
    
    # Email Section
    st.markdown("---")
    st.header("📧 Send Email Report")
    
    # Department to warehouse mapping based on line 59 warehouses
    # Warehouses: G_Active_1, G_Active_2, G_MD_1, G_MD_2, HGBU_Extra, Pre_Ship_1, Pre_Ship_2, 
    #             WIPLines1, WIPLines2, WIP_Cut_1, WIP_Emb_1, WIP_P1, WIP_Pri_1, 
    #             WIP_Sew_1, WIP_Sew_2, WIP_Sew_P1, WIP_Sew_P2, PF_Active
    
    department_warehouse_mapping = {
        "Garment Active (G_Active)": {
            "email": "garment.active@company.com",
            "warehouses": ['G_Active_1', 'G_Active_2']
        },
        "Garment MD (G_MD)": {
            "email": "garment.md@company.com",
            "warehouses": ['G_MD_1', 'G_MD_2']
        },
        "Pre-Shipment": {
            "email": "preshipment@company.com",
            "warehouses": ['Pre_Ship_1', 'Pre_Ship_2']
        },
        "WIP Lines": {
            "email": "wiplines@company.com",
            "warehouses": ['WIPLines1', 'WIPLines2']
        },
        "WIP Sewing": {
            "email": "wipsewing@company.com",
            "warehouses": ['WIP_Sew_1', 'WIP_Sew_2', 'WIP_Sew_P1', 'WIP_Sew_P2']
        },
        "WIP Cutting & Print": {
            "email": "wipcutting@company.com",
            "warehouses": ['WIP_Cut_1', 'WIP_Pri_1', 'WIP_P1']
        },
        "WIP Embroidery": {
            "email": "wipembroidery@company.com",
            "warehouses": ['WIP_Emb_1']
        },
        "Fabric Department (PF_Active)": {
            "email": "fabric@company.com",
            "warehouses": ['PF_Active']
        },
        "HGBU Extra": {
            "email": "hgbu@company.com",
            "warehouses": ['HGBU_Extra']
        },
        "All Warehouses": {
            "email": "management@company.com",
            "warehouses": list(df['Warehouse'].unique()) + (['PF_Active'] if 'PF_Active' in df2['Ware House'].unique() else [])
        }
    }
    
    col_email1, col_email2 = st.columns([1, 1])

## 📧 Email Configuration & File Preview

This section provides the email composition interface with automatic file attachment preview:

---

## 📝 Left Column - Email Configuration (col_email1)

### Department Selection
```python
selected_department = st.selectbox("Select Department", list(department_warehouse_mapping.keys()))
department_warehouses = department_warehouse_mapping[selected_department]["warehouses"]
```
**Features:**
- Dropdown menu with 10 department options
- Automatically maps to corresponding warehouses
- Dynamic warehouse list based on selection

**Available Departments:**
- Garment Active, Garment MD, Pre-Shipment, WIP Lines, WIP Sewing
- WIP Cutting & Print, WIP Embroidery, Fabric Department, HGBU Extra, All Warehouses

### Email Recipients
```python
recipient_email = st.text_input("Recipient Email Address", placeholder="recipient@company.com")
```
- Text input for destination email address
- Placeholder text for guidance
- Required for sending email

### Warehouse Information Display
```python
st.info(f"📦 Warehouses: {', '.join(department_warehouses)}")
```
- Shows warehouses included in selected department
- Blue info box for visibility
- Helps user confirm correct department selection

### Sender Credentials
```python
sender_email = st.text_input("Your Gmail Address", placeholder="your.email@gmail.com")
```
- Gmail address for authentication
- Required for OAuth2 authentication
- Must be authorized Gmail account

### Email Subject Line
```python
email_subject = st.text_input("Email Subject", 
                               value=f"Warehouse Stock Report - {selected_department} - {current_date}")
```
**Auto-generated format:**
- `Warehouse Stock Report - {Department} - {Date}`
- Example: "Warehouse Stock Report - WIP Sewing - 26-12-2024"
- User can edit before sending

---

## 📎 Right Column - Files to be Sent (col_email2)

### Automatic File Determination

**Process:**
1. Checks current filter type (Days or Statistical)
2. Iterates through department's warehouses
3. Applies current filters to determine if data exists
4. Builds list of files to attach

### Days Filter Mode
```python
if filter_type == "Days":
    st.write(f"**Filter: Days ({', '.join(selected_days)})**")
    filter_suffix = "DaysFilter"
```
**Behavior:**
- Displays selected day categories
- Checks each warehouse for items matching `selected_days`
- Adds warehouse to attachment list if data exists
- Counts total items across all files

**File Naming:**
- Format: `{warehouse}_{date}_DaysFilter.xlsx`
- Example: `WIP_Sew_1_26-12-2024_DaysFilter.xlsx`

### Statistical Filter Mode
```python
else:  # Statistical
    st.write("**Filter: Critical Items**")
    filter_suffix = "Critical"
```
**Behavior:**
- Shows "Critical Items" label
- Checks each warehouse for critical items
- Uses `df['Critical']` boolean column
- Only includes warehouses with critical items

**File Naming:**
- Format: `{warehouse}_{date}_Critical.xlsx`
- Example: `G_Active_1_26-12-2024_Critical.xlsx`

### PF_Active Special Handling
```python
if warehouse == 'PF_Active':
    if 'PF_Active' in df2['Ware House'].unique():
        warehouse_df = df2[(df2['Ware House'] == warehouse) & ...]
```
- Uses `df2` (fabric stock) instead of `df`
- Different column structure (Lot No, Style-color, Gramaj)
- Checks existence before filtering

---

## 📊 File Preview Display

### Success State (Files Available)
```python
if files_to_send:
    st.success(f"✅ **{len(files_to_send)} file(s) ready to send**")
```

**Displays:**
1. **Success message** - Green banner with file count
2. **File list** - Bulleted list of attachment filenames
3. **Total Items metric** - Large number showing total item count

**Example Output:**
```
✅ 3 file(s) ready to send

Files that will be attached:
• WIP_Sew_1_26-12-2024_Critical.xlsx
• WIP_Sew_2_26-12-2024_Critical.xlsx
• WIP_Sew_P1_26-12-2024_Critical.xlsx

Total Items
    127
```

### Warning State (No Files)
```python
else:
    st.warning("⚠️ No data available for selected department with current filters")
```
- Yellow/orange warning banner
- Indicates no items match current filter criteria
- User can adjust filters or select different department

---

## 🔄 Dynamic Updates

All file preview information updates automatically when:
- User selects different department
- Filter type changes (Days ↔ Statistical)
- Day categories are modified (in Days mode)
- Different warehouse selected in sidebar

This ensures the email preview always reflects the current state of filters and selections.

In [ ]:
if st.session_state.processed:    

    with col_email1:
        st.subheader("Email Configuration")
        
        # Department selection
        selected_department = st.selectbox("Select Department", list(department_warehouse_mapping.keys()))
        department_warehouses = department_warehouse_mapping[selected_department]["warehouses"]
        
        # Recipient email input
        recipient_email = st.text_input("Recipient Email Address", placeholder="recipient@company.com")
        
        st.info(f"📦 Warehouses: {', '.join(department_warehouses)}")
        
        # Sender credentials
        sender_email = st.text_input("Your Gmail Address", placeholder="your.email@gmail.com")
        
        # Email subject
        email_subject = st.text_input("Email Subject", 
                                     value=f"Warehouse Stock Report - {selected_department} - {current_date}")
    
    with col_email2:
        st.subheader("Files to be Sent")
        
        # Automatically determine files based on department warehouses and current filters
        files_to_send = []
        total_items = 0
        
        if filter_type == "Days":
            st.write(f"**Filter: Days ({', '.join(selected_days)})**")
            filter_suffix = "DaysFilter"
            
            for warehouse in department_warehouses:
                if warehouse == 'PF_Active':
                    if 'PF_Active' in df2['Ware House'].unique():
                        warehouse_df = df2[(df2['Ware House'] == warehouse) & (df2['days cat'].isin(selected_days))]
                        if not warehouse_df.empty:
                            files_to_send.append(warehouse)
                            total_items += len(warehouse_df)
                else:
                    if warehouse in df['Warehouse'].unique():
                        warehouse_df = df[(df['Warehouse'] == warehouse) & (df['days cat'].isin(selected_days))]
                        if not warehouse_df.empty:
                            files_to_send.append(warehouse)
                            total_items += len(warehouse_df)
        else:  # Statistical
            st.write("**Filter: Critical Items**")
            filter_suffix = "Critical"
            
            for warehouse in department_warehouses:
                if warehouse == 'PF_Active':
                    if 'PF_Active' in df2['Ware House'].unique():
                        warehouse_df = df2[(df2['Ware House'] == warehouse) & (df2['Critical'])]
                        if not warehouse_df.empty:
                            files_to_send.append(warehouse)
                            total_items += len(warehouse_df)
                else:
                    if warehouse in df['Warehouse'].unique():
                        warehouse_df = df[(df['Warehouse'] == warehouse) & (df['Critical'])]
                        if not warehouse_df.empty:
                            files_to_send.append(warehouse)
                            total_items += len(warehouse_df)
        
        if files_to_send:
            st.success(f"✅ **{len(files_to_send)} file(s) ready to send**")
            st.write("**Files that will be attached:**")
            for file in files_to_send:
                st.write(f"• {file}_{current_date}_{filter_suffix}.xlsx")
            st.metric("Total Items", total_items)
        else:
            st.warning("⚠️ No data available for selected department with current filters")


## 📧 Email Template Preview & Dynamic Content Generation

This section creates a professional email template with dynamically generated content based on current filters and data:

---

## 🔍 Filter Description Logic

### Days Filter - Smart Description
```python
if filter_type == "Days":
    late_categories = [cat for cat in selected_days if cat in ['61 - 90 days', '91 - 180 days', '180+ days']]
    if late_categories:
        filter_description = "Projects that stayed over 60 days"
    else:
        filter_description = f"Days Filter: {', '.join(selected_days)}"
```

**Behavior:**
- **Late Categories Detected** (61+ days selected):
  - Description: "Projects that stayed over 60 days"
  - Focus on aging inventory problem areas
- **Other Categories Only** (0-60 days):
  - Description: "Days Filter: 0-15 days, 16-30 days, 31-60 days"
  - Lists specific categories selected

### Statistical Filter - Simple Description
```python
else:
    filter_description = "Critical projects"
```
- Concise label for statistically significant items
- Indicates items exceeding 95% confidence interval threshold

---

## 🏆 Top 3 Projects Identification

### Purpose
Highlights the **3 most concerning projects** for management attention based on longest time in warehouse.

### Days Filter - Most Late Projects

**Data Collection:**
```python
all_data_frames = []
for warehouse in files_to_send:
    if warehouse == 'PF_Active':
        warehouse_df = df2[...][['Project', 'number of days']].copy()
    else:
        warehouse_df = df[...][['Project', 'number of days']].copy()
    all_data_frames.append(warehouse_df)
```

**Process:**
1. Iterates through warehouses in selected department
2. Extracts **Project** and **number of days** columns
3. Handles PF_Active separately (uses `df2` fabric stock)
4. Combines all department data into single DataFrame

**Ranking:**
```python
combined_df = pd.concat(all_data_frames, ignore_index=True)
top_3 = combined_df.nlargest(3, 'number of days')
```
- Merges all warehouse data
- Sorts by `number of days` (descending)
- Takes top 3 projects with longest time

**Output Format:**
```
Top 3 Most Late Projects:
  1. PROJECT_ABC - 245 days
  2. PROJECT_XYZ - 198 days
  3. PROJECT_DEF - 167 days
```

### Statistical Filter - Most Critical Projects

**Same Logic, Different Filter:**
```python
warehouse_df = df[(df['Warehouse'] == warehouse) & (df['Critical'])]
```
- Uses `Critical` boolean column instead of day categories
- Identifies projects exceeding statistical threshold
- Same ranking and output format

**Output:**
```
Top 3 Most Critical Projects:
  1. PROJECT_GHI - 312 days
  2. PROJECT_JKL - 278 days
  3. PROJECT_MNO - 251 days
```

### Edge Cases
- **No files to send** → Empty `top_projects_text` (no section added)
- **Fewer than 3 projects** → Shows only available projects
- **Empty DataFrames** → Skips concatenation, no top 3 section

---

## 📝 Email Body Template

### Dynamic Template Structure
```python
email_body_template = f"""
Dear {selected_department} Team,

Please find attached the Warehouse Stock Report for your review.

Report Details:
- Report Date: {current_date}
- Department: {selected_department}
- Filter Applied: {filter_description}
- Number of Files: {len(files_to_send)}
- Total Items: {total_items}
- Warehouses Included: {', '.join(files_to_send) if files_to_send else 'None'}{top_projects_text}

The attached Excel file(s) contain detailed information about stock items based on the applied filters.

Please review the data and take necessary actions as required.

Best regards,
Bassem
Planning Department
"""
```

### Template Variables

| Variable | Description | Example |
|----------|-------------|---------|
| `{selected_department}` | Department name from dropdown | "WIP Sewing" |
| `{current_date}` | Analysis reference date | "26-12-2024" |
| `{filter_description}` | Dynamic filter summary | "Projects that stayed over 60 days" |
| `{len(files_to_send)}` | Count of Excel attachments | 4 |
| `{total_items}` | Total items across all files | 127 |
| `{', '.join(files_to_send)}` | Warehouse list | "WIP_Sew_1, WIP_Sew_2, WIP_Sew_P1, WIP_Sew_P2" |
| `{top_projects_text}` | Top 3 projects section (optional) | See formatting above |

### Example Output

**Days Filter (Late Categories):**
```
Dear WIP Sewing Team,

Please find attached the Warehouse Stock Report for your review.

Report Details:
- Report Date: 26-12-2024
- Department: WIP Sewing
- Filter Applied: Projects that stayed over 60 days
- Number of Files: 4
- Total Items: 127
- Warehouses Included: WIP_Sew_1, WIP_Sew_2, WIP_Sew_P1, WIP_Sew_P2

Top 3 Most Late Projects:
  1. PROJECT_ABC - 245 days
  2. PROJECT_XYZ - 198 days
  3. PROJECT_DEF - 167 days

The attached Excel file(s) contain detailed information about stock items based on the applied filters.

Please review the data and take necessary actions as required.

Best regards,
Bassem
Planning Department
```

**Statistical Filter:**
```
Dear Fabric Department (PF_Active) Team,

Please find attached the Warehouse Stock Report for your review.

Report Details:
- Report Date: 26-12-2024
- Department: Fabric Department (PF_Active)
- Filter Applied: Critical projects
- Number of Files: 1
- Total Items: 23
- Warehouses Included: PF_Active

Top 3 Most Critical Projects:
  1. FABRIC_101 - 312 days
  2. FABRIC_202 - 278 days
  3. FABRIC_303 - 251 days

...
```

---

## 📄 Template Preview Display

### Text Area Component
```python
st.text_area("Email Body", email_body_template, height=300, disabled=True)
```

**Features:**
- **Height**: 300px - Large enough to show full template without scrolling
- **Disabled**: True - Read-only preview (not editable)
- **Label**: "Email Body" - Clear indication of content purpose
- **Live Updates**: Changes dynamically when:
  - Different department selected
  - Filter type switches
  - Day categories modified
  - Files availability changes

**Purpose:**
- Allows user to review email content before sending
- Ensures accuracy of dynamic content
- Provides transparency about what recipients will receive

---

## ✅ Credentials Validation

### Send Button State Control
```python
st.markdown("---")
credentials_valid = bool(sender_email and recipient_email)
```

**Validation Logic:**
- **sender_email** → Must be provided (Gmail address for OAuth)
- **recipient_email** → Must be provided (destination address)
- **Both required** → Button disabled if either is missing

**Button State (next section):**
```python
disabled=not (credentials_valid and files_to_send)
```
- **Enabled** → Only when credentials valid AND files available
- **Disabled** → Missing credentials OR no data to send

**Visual Separator:**
- `st.markdown("---")` adds horizontal line
- Separates preview from action button
- Clear visual boundary between review and execution

---

## 🔄 Dynamic Content Updates

All email template content updates in real-time when:
1. **Department changes** → New warehouse list, recipient, top 3 projects
2. **Filter type switches** → Different filter description and critical logic
3. **Day categories modified** → Updated late categories detection
4. **Files become available/unavailable** → Top 3 section appears/disappears
5. **Data changes** → Updated total items and file counts

This ensures the email preview always reflects the current selection state.

In [ ]:
if st.session_state.processed:
    
    # Email template
    st.subheader("Email Template Preview")
    
    # Determine filter description based on filter type
    if filter_type == "Days":
        # Check if any late categories (over 60 days) are selected
        late_categories = [cat for cat in selected_days if cat in ['61 - 90 days', '91 - 180 days', '180+ days']]
        if late_categories:
            filter_description = "Projects that stayed over 60 days"
        else:
            filter_description = f"Days Filter: {', '.join(selected_days)}"
    else:
        filter_description = "Critical projects"
    
    # Get top 3 projects
    top_projects_text = ""
    if files_to_send:
        if filter_type == "Days":
            # Get top 3 most late projects (longest time)
            all_data_frames = []
            for warehouse in files_to_send:
                if warehouse == 'PF_Active':
                    if 'PF_Active' in df2['Ware House'].unique():
                        warehouse_df = df2[(df2['Ware House'] == warehouse) & (df2['days cat'].isin(selected_days))][['Project', 'number of days']].copy()
                        all_data_frames.append(warehouse_df)
                else:
                    if warehouse in df['Warehouse'].unique():
                        warehouse_df = df[(df['Warehouse'] == warehouse) & (df['days cat'].isin(selected_days))][['Project', 'number of days']].copy()
                        all_data_frames.append(warehouse_df)
            
            if all_data_frames:
                combined_df = pd.concat(all_data_frames, ignore_index=True)
                top_3 = combined_df.nlargest(3, 'number of days')
                top_projects_text = "\nTop 3 Most Late Projects:\n"
                for idx, row in top_3.iterrows():
                    top_projects_text += f"  {idx+1}. {row['Project']} - {int(row['number of days'])} days\n"
        else:  # Critical
            # Get top 3 critical projects (longest time)
            all_data_frames = []
            for warehouse in files_to_send:
                if warehouse == 'PF_Active':
                    if 'PF_Active' in df2['Ware House'].unique():
                        warehouse_df = df2[(df2['Ware House'] == warehouse) & (df2['Critical'])][['Project', 'number of days']].copy()
                        all_data_frames.append(warehouse_df)
                else:
                    if warehouse in df['Warehouse'].unique():
                        warehouse_df = df[(df['Warehouse'] == warehouse) & (df['Critical'])][['Project', 'number of days']].copy()
                        all_data_frames.append(warehouse_df)
            
            if all_data_frames:
                combined_df = pd.concat(all_data_frames, ignore_index=True)
                top_3 = combined_df.nlargest(3, 'number of days')
                top_projects_text = "\nTop 3 Most Critical Projects:\n"
                for idx, row in top_3.iterrows():
                    top_projects_text += f"  {idx+1}. {row['Project']} - {int(row['number of days'])} days\n"
    
    email_body_template = f"""
Dear {selected_department} Team,

Please find attached the Warehouse Stock Report for your review.

Report Details:
- Report Date: {current_date}
- Department: {selected_department}
- Filter Applied: {filter_description}
- Number of Files: {len(files_to_send)}
- Total Items: {total_items}
- Warehouses Included: {', '.join(files_to_send) if files_to_send else 'None'}{top_projects_text}

The attached Excel file(s) contain detailed information about stock items based on the applied filters.

Please review the data and take necessary actions as required.

Best regards,
Bassem
Planning Department
"""
    
    st.text_area("Email Body", email_body_template, height=300, disabled=True)
    
    # Send email button
    st.markdown("---")
    
    # Check credentials
    credentials_valid = bool(sender_email and recipient_email)


## 📤 Email Sending & Gmail API Integration

This section handles the complete email sending workflow using Gmail API with OAuth2 authentication:

---

## 🔘 Send Email Button

### Button Configuration
```python
st.button("📤 Send Email", type="primary", disabled=not (credentials_valid and files_to_send))
```

**Button States:**
- **Enabled** (Blue primary button):
  - `credentials_valid` = True (both sender and recipient emails provided)
  - `files_to_send` list is not empty (data available)
- **Disabled** (Grayed out):
  - Missing sender or recipient email
  - No files to attach (no matching data)

### Validation Checks

**Pre-send Validation:**
```python
if not sender_email:
    st.error("Please provide your email address")
elif not recipient_email:
    st.error("Please provide recipient email address")
elif not files_to_send:
    st.error("No data available for the selected department with current filters")
```

**Validation Order:**
1. Sender email presence check
2. Recipient email presence check  
3. Files availability check

Each displays a red error message if condition fails.

---

## 📧 Email Message Construction

### MIME Multipart Message
```python
msg = MIMEMultipart()
msg['From'] = sender_email
msg['To'] = recipient_email
msg['Subject'] = email_subject
```

**Structure:**
- **From** - Gmail address for authentication
- **To** - Recipient address (department contact)
- **Subject** - Auto-generated or user-edited subject line

### Email Body Attachment
```python
msg.attach(MIMEText(email_body_template, 'plain'))
```
- Attaches the dynamically generated email template
- Plain text format (not HTML)
- Contains department info, filter details, top 3 projects

---

## 📎 Excel File Generation & Attachment

### File Creation Loop
```python
for warehouse_name in files_to_send:
```
Iterates through each warehouse that has matching data for the selected department.

### Data Filtering Logic

**Days Filter Mode:**
```python
if filter_type == "Days":
    if warehouse_name == 'PF_Active':
        warehouse_data = df2[(df2['Ware House'] == warehouse_name) & 
                            (df2['days cat'].isin(selected_days))][
            ['Project', 'Lot No', 'Style-color', 'Gramaj', 'last transaction date', 'number of days']
        ].copy()
    else:
        warehouse_data = df[(df['Warehouse'] == warehouse_name) & 
                           (df['days cat'].isin(selected_days))][
            ['Project', 'Color', 'Size', 'Quantity', 'Customer', 'Last Movement Date', 'number of days']
        ].copy()
```

**Behavior:**
- **PF_Active** - Extracts from `df2` (fabric stock) with fabric-specific columns
- **Other Warehouses** - Extracts from `df` (stock source) with standard columns
- Applies selected day categories filter
- Includes `number of days` for aging visibility

**Statistical Filter Mode:**
```python
else:  # Statistical
    if warehouse_name == 'PF_Active':
        warehouse_data = df2[(df2['Ware House'] == warehouse_name) & (df2['Critical'])]
    else:
        warehouse_data = df[(df['Warehouse'] == warehouse_name) & (df['Critical'])]
```

**Behavior:**
- Uses `Critical` boolean column instead of day categories
- Same dual-source logic (df2 for PF_Active, df for others)
- Same column structure as Days filter

### Excel Conversion
```python
excel_buffer = io.BytesIO()
with pd.ExcelWriter(excel_buffer, engine='openpyxl') as writer:
    warehouse_data.to_excel(writer, sheet_name='Stock Report', index=False)
excel_buffer.seek(0)
```

**Process:**
1. Creates in-memory buffer (no temporary files)
2. Uses `openpyxl` engine for .xlsx format
3. Sheet name: "Stock Report"
4. `index=False` - Excludes pandas index column
5. Resets buffer position to start for reading

### MIME Attachment Creation
```python
filename = f"{warehouse_name}_{current_date}_{filter_suffix}.xlsx"

part = MIMEBase('application', 'vnd.openxmlformats-officedocument.spreadsheetml.sheet')
part.set_payload(excel_buffer.read())
encoders.encode_base64(part)
part.add_header('Content-Disposition', f'attachment; filename={filename}')
msg.attach(part)
```

**Steps:**
1. **Filename format** - `WIP_Sew_1_26-12-2024_DaysFilter.xlsx` or `Critical` suffix
2. **MIME type** - Excel 2007+ format (.xlsx)
3. **Payload** - Reads Excel buffer contents
4. **Encoding** - Base64 for email transmission
5. **Disposition** - Marks as downloadable attachment
6. **Attachment** - Adds to message object

---

## 🔐 Gmail API Authentication (OAuth2)

### Required Imports
```python
import os
from google_auth_oauthlib.flow import InstalledAppFlow
import pickle
```

### OAuth2 Scopes
```python
SCOPES = ['https://www.googleapis.com/auth/gmail.send']
```
- Minimal scope for sending emails only
- Does not grant read access to inbox
- Follows principle of least privilege

### Token Management

**Check for Saved Credentials:**
```python
if os.path.exists('token.pickle'):
    with open('token.pickle', 'rb') as token:
        creds = pickle.load(token)
```
- Looks for previously saved credentials
- Avoids re-authentication on every send
- Credentials persist across sessions

**Credential Validation:**
```python
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
```
- Checks if credentials exist and are valid
- Automatically refreshes expired tokens
- Uses refresh token to avoid manual re-authentication

### Initial Authentication Flow
```python
credentials_json_path = r"your_credentials_json.json"

if os.path.exists(credentials_json_path):
    flow = InstalledAppFlow.from_client_secrets_file(credentials_json_path, SCOPES)
    creds = flow.run_local_server(port=0)
    
    # Save credentials for next run
    with open('token.pickle', 'wb') as token:
        pickle.dump(creds, token)
```

**Process:**
1. **Credentials File** - Google Cloud OAuth2 client credentials JSON
2. **OAuth Flow** - Opens browser for user authorization
3. **Local Server** - Runs temporary web server (port 0 = random available port)
4. **Authorization** - User grants permissions in browser
5. **Token Save** - Stores credentials for future use

**Error Handling:**
```python
else:
    st.error("Credentials file not found at the specified path")
    st.stop()
```
- Displays error if credentials JSON missing
- Halts execution (`st.stop()`) to prevent further errors

---

## 📨 Gmail API Service & Sending

### Build Gmail Service
```python
service = build('gmail', 'v1', credentials=creds)
```
- Constructs Gmail API client
- Version 1 of Gmail API
- Uses authenticated credentials

### Message Encoding
```python
raw_message = base64.urlsafe_b64encode(msg.as_bytes()).decode('utf-8')
message = {'raw': raw_message}
```

**Process:**
1. Converts MIME message to bytes
2. Base64 URL-safe encoding (required by Gmail API)
3. Decodes to UTF-8 string
4. Wraps in dictionary with 'raw' key

### Send Email
```python
service.users().messages().send(userId='me', body=message).execute()
```

**Parameters:**
- `userId='me'` - Sends from authenticated user (sender_email)
- `body=message` - Contains encoded email with attachments
- `.execute()` - Performs the API call

### Success Confirmation
```python
st.success(f"✅ Email sent successfully to {selected_department} ({recipient_email})")
st.balloons()
```
- Green success message with department and recipient
- Celebratory balloon animation
- Confirms completion to user

---

## ⚠️ Error Handling

### SMTP Authentication Error
```python
except smtplib.SMTPAuthenticationError:
    st.error("❌ Authentication failed. Please check your email and app password. 
             For Gmail, make sure you're using an App Password, not your regular password.")
```
- Catches authentication failures (should be rare with OAuth2)
- Provides specific guidance for Gmail App Passwords
- Displayed as red error message

### Generic Exception Handler
```python
except Exception as e:
    st.error(f"❌ Failed to send email: {str(e)}\n\nPlease check your credentials and try again.")
```

**Catches:**
- Network errors
- API quota exceeded errors
- Invalid credentials
- File attachment errors
- Any unexpected failures

**Displays:**
- Error icon (❌)
- Error message text
- Exception details for debugging
- Suggestion to check credentials

---

## 🏠 Initial State Display

### Empty State Handling
```python
else:
    st.info("Please upload both Stock Source and Fabric Stock files to begin analysis.")
```

**When Displayed:**
- User first loads the application
- Before any files are uploaded
- After clicking "Reset and Upload New Files"
- When `st.session_state.processed` is False

**Purpose:**
- Clear instruction for next step
- Blue info banner (friendly, non-intrusive)
- Guides user to upload required files
- Prevents confusion with empty dashboard

---

## 🔄 Complete Workflow Summary

**User Journey:**
1. Upload Stock Source and Fabric Stock files
2. Data is processed automatically
3. View dashboard and apply filters
4. Navigate to email section
5. Select department from dropdown
6. Enter recipient and sender emails
7. Review file preview (automatic)
8. Review email template preview
9. Click "Send Email" button
10. **First time only:** Browser opens for Google OAuth2 authorization
11. Credentials saved to `token.pickle`
12. Excel files generated in memory for each warehouse
13. Files attached to email
14. Email sent via Gmail API
15. Success confirmation with balloons animation

**Subsequent Sends:**
- Skip OAuth step (uses saved token)
- Automatic token refresh if expired
- Seamless user experience

---

## 📋 Prerequisites for Email Sending

**Required Setup:**
1. **Google Cloud Project** - Create at console.cloud.google.com
2. **Gmail API Enabled** - Enable in API Library
3. **OAuth2 Credentials** - Create Desktop App credentials
4. **Credentials JSON** - Download and save as `your_credentials_json.json`
5. **File Path Update** - Update `credentials_json_path` variable with actual path
6. **First Run** - Browser authorization on first send
7. **Token Saved** - `token.pickle` created automatically

**Optional Dependencies:**
- Already installed: `google-auth`, `google-api-python-client` (from requirements)
- Already installed: `google-auth-oauthlib` (for OAuth flow)

In [ ]:
if st.session_state.processed:
    
    if st.button("📤 Send Email", type="primary", disabled=not (credentials_valid and files_to_send)):
        if not sender_email:
            st.error("Please provide your email address")
        elif not recipient_email:
            st.error("Please provide recipient email address")
        elif not files_to_send:
            st.error("No data available for the selected department with current filters")
        else:
            with st.spinner("Preparing and sending email..."):
                try:
                    # Create message
                    msg = MIMEMultipart()
                    msg['From'] = sender_email
                    msg['To'] = recipient_email
                    msg['Subject'] = email_subject
                    
                    # Attach email body
                    msg.attach(MIMEText(email_body_template, 'plain'))
                    
                    # Create Excel files for warehouses in this department
                    for warehouse_name in files_to_send:
                        # Get the filtered data for this warehouse
                        if filter_type == "Days":
                            if warehouse_name == 'PF_Active':
                                warehouse_data = df2[(df2['Ware House'] == warehouse_name) & 
                                                    (df2['days cat'].isin(selected_days))][
                                    ['Project', 'Lot No', 'Style-color', 'Gramaj', 'last transaction date', 'number of days']
                                ].copy()
                            else:
                                warehouse_data = df[(df['Warehouse'] == warehouse_name) & 
                                                   (df['days cat'].isin(selected_days))][
                                    ['Project', 'Color', 'Size', 'Quantity', 'Customer', 'Last Movement Date', 'number of days']
                                ].copy()
                        else:  # Statistical
                            if warehouse_name == 'PF_Active':
                                warehouse_data = df2[(df2['Ware House'] == warehouse_name) & 
                                                    (df2['Critical'])][
                                    ['Project', 'Lot No', 'Style-color', 'Gramaj', 'last transaction date', 'number of days']
                                ].copy()
                            else:
                                warehouse_data = df[(df['Warehouse'] == warehouse_name) & 
                                                   (df['Critical'])][
                                    ['Project', 'Color', 'Size', 'Quantity', 'Customer', 'Last Movement Date', 'number of days']
                                ].copy()
                        
                        # Convert to Excel
                        excel_buffer = io.BytesIO()
                        with pd.ExcelWriter(excel_buffer, engine='openpyxl') as writer:
                            warehouse_data.to_excel(writer, sheet_name='Stock Report', index=False)
                        excel_buffer.seek(0)
                        
                        # Attach file
                        filename = f"{warehouse_name}_{current_date}_{filter_suffix}.xlsx"
                        
                        part = MIMEBase('application', 'vnd.openxmlformats-officedocument.spreadsheetml.sheet')
                        part.set_payload(excel_buffer.read())
                        encoders.encode_base64(part)
                        part.add_header('Content-Disposition', f'attachment; filename={filename}')
                        msg.attach(part)
                    
                    # Send email based on selected method
                    # Send via Google Cloud Gmail API
                    import os
                    from google_auth_oauthlib.flow import InstalledAppFlow
                    import pickle
                    
                    SCOPES = ['https://www.googleapis.com/auth/gmail.send']
                    creds = None
                    
                    # Check for saved token
                    if os.path.exists('token.pickle'):
                        with open('token.pickle', 'rb') as token:
                            creds = pickle.load(token)
                    
                    # If no valid credentials, authenticate
                    if not creds or not creds.valid:
                        if creds and creds.expired and creds.refresh_token:
                            creds.refresh(Request())
                        else:
                            credentials_json_path = r"your_credentials_json.json"
                            
                            if os.path.exists(credentials_json_path):
                                flow = InstalledAppFlow.from_client_secrets_file(
                                    credentials_json_path, SCOPES)
                                creds = flow.run_local_server(port=0)
                                
                                # Save credentials for next run
                                with open('token.pickle', 'wb') as token:
                                    pickle.dump(creds, token)
                            else:
                                st.error("Credentials file not found at the specified path")
                                st.stop()
                    
                    # Build Gmail API service
                    service = build('gmail', 'v1', credentials=creds)
                    
                    # Encode message
                    raw_message = base64.urlsafe_b64encode(msg.as_bytes()).decode('utf-8')
                    message = {'raw': raw_message}
                    
                    # Send message
                    service.users().messages().send(userId='me', body=message).execute()
                    
                    st.success(f"✅ Email sent successfully to {selected_department} ({recipient_email})")
                    st.balloons()
                    
                except smtplib.SMTPAuthenticationError:
                    st.error("❌ Authentication failed. Please check your email and app password. For Gmail, make sure you're using an App Password, not your regular password.")
                except Exception as e:
                    st.error(f"❌ Failed to send email: {str(e)}\n\nPlease check your credentials and try again.")

else:
    st.info("Please upload both Stock Source and Fabric Stock files to begin analysis.")